# valdata testing

## Packages and setup

In [1]:
# !pip install ipykernel
# !pip install pandas==1.5.2
# !pip install numpy==1.22.4
# !pip install pyspark==3.1.1
# !pip install pytz
# !pip install tabulate
# !pip install importlib

In [2]:
# !pip freeze

In [3]:
import importlib
import valdata as vd
importlib.reload(vd)

from time import sleep

import pandas as pd
import numpy as np

from pyspark.sql import SparkSession
from pyspark.sql import DataFrame as SparkDataFrame

In [4]:
# pd.set_option('display.max_rows', 5000)
# pd.set_option('display.max_columns', 5000)
# pd.set_option('display.max_colwidth', None)
# pd.set_option('display.width', 1000)

In [5]:
spark = SparkSession.builder.appName("valdata_testing").getOrCreate()

## DBs

In [6]:
citizens_dict = {
  'Name': ['Alice', 'Alice', 'Bob', 'Charlie', 'David'],
  'Age': [24, 27, 22, 32, 44],
  'City': ['New York', 'New York', 'Los Angeles', 'Chicago', 'Houston']
  }
citizens2_dict = {
  'Name': ['Juan', 'Camilo', 'Roberto', 'Carlos', 'David'],
  'Age': [20, 57, 57, 32, 34],
  'City': ['Bogota', 'Cali', 'Medellin', 'Bogota', 'Bogota']
  }
citizensN_dict = {
  'name': ['Alice', 'Bob', 'Charlie', 'Jules', 'Froy', 'Mack', 'Kim', 'Ron', 'Juan', 'Michael', 'Mister'],
  'age': [25, 30, 35, 20, 20, 20, 30, 30, 30, 24, 23],
  'city': ['NewYork', 'LosAngeles', 'Chicago', 'NewYork', 'LosAngeles', 'Chicago', 'NewYork', 'LosAngeles', 'Chicago', 'Bogota', 'Lima'],
  'number': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
}
citizensNN_dict = {
  'name': ['Alice', 'Bob', 'Charlie', 'Jules', 'Froy', 'Mack', 'Kim', 'Ron', 'Juan', 'Michael', 'Mister', 'Rolando'],
  'age': [25, 30, 35, 20, 20, 20, 30, 30, 30, 24, 23, 15],
  'city': ['NewYork', 'LosAngeles', 'Chicago', 'NewYork', 'LosAngeles', 'Chicago', 'NewYork', 'LosAngeles', 'Chicago', 'Bogota', 'Lima', 'Aragua'],
  'number': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13],
  'number2': [2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 26]
}

In [7]:
citizens_df = pd.DataFrame(citizens_dict)
citizens2_df = pd.DataFrame(citizens2_dict)
citizensN_pq = spark.createDataFrame(pd.DataFrame(citizensN_dict))
citizensNN_pq = spark.createDataFrame(pd.DataFrame(citizensNN_dict))

c:\Users\godoy\Downloads\WebDev\valdata\venv\lib\site-packages\pyspark\sql\pandas\conversion.py:331: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [8]:
titanic_df = pd.read_csv("https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv")
# Spark expects each field to have a consistent data type, in this case this have (nan, string)
titanic_df['Cabin'] = titanic_df['Cabin'].astype(str) 
titanic_df['Embarked'] = titanic_df['Embarked'].astype(str)
titanic_df_slice1 = titanic_df.iloc[:, :8]
titanic_df_slice2 = titanic_df.iloc[:, 5:9]
titanic_df_slice2.rename(columns={titanic_df_slice2.columns[1]: "this_is_a_new_column"}, inplace=True)
titanic_df_copy = titanic_df.copy()
titanic_df_copy.at[500, 'Cabin'] = 'INPUTTED REGISTER'

titanic_pq = spark.createDataFrame(titanic_df)
titanic_pq_slice1 = spark.createDataFrame(titanic_df_slice1)
titanic_pq_slice2 = spark.createDataFrame(titanic_df_slice2)
titanic_pq_copy = spark.createDataFrame(titanic_df_copy)

c:\Users\godoy\Downloads\WebDev\valdata\venv\lib\site-packages\pyspark\sql\pandas\conversion.py:331: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [9]:
pd_series = pd.Series([1, 2, 3])
np_array = np.array([1, 2, 3])
py_list = [1, 2, 3]

## Functions

### - validation

In [10]:
vd.now()

'Nov-10-24 06:42:23 PM'

In [11]:
vd.tms_0()
print('Starting operation')
sleep(20)
print('Finishing operation')
vd.tms_1()

Starting operation
Finishing operation
Execution time: 00 - 00:00:20 [Finished at: Nov-10-24 06:42:43 PM]


In [12]:
vd.get_overview(citizensN_pq)

Data shape: Variables: 4
            Registers: 11
Type: <class 'pyspark.sql.dataframe.DataFrame'>
Data:
+-------+---+----------+------+
|   name|age|      city|number|
+-------+---+----------+------+
|  Alice| 25|   NewYork|     1|
|    Bob| 30|LosAngeles|     2|
|Charlie| 35|   Chicago|     3|
|  Jules| 20|   NewYork|     4|
|   Froy| 20|LosAngeles|     5|
+-------+---+----------+------+
only showing top 5 rows

Composition:
+------------+----------------+--------------------------------------------------------------------------+
| Variable   |   Unique Count | Unique Values                                                            |
|------------+----------------+--------------------------------------------------------------------------|
| name       |             11 | [Juan, Froy, Mack, Ron, Kim, Michael, Mister, Jules, Bob, Alice, 1 more] |
| age        |              6 | [30, 20, 35, 24, 25, 23]                                                 |
| city       |              5 | [L

In [13]:
vd.get_overview(titanic_df)

Data shape: Variables: 12
            Registers: 891
Type: <class 'pandas.core.frame.DataFrame'>
Data:
+---------------+------------+----------+-----------------------------------------------------+--------+-------+---------+---------+------------------+---------+---------+------------+
|   PassengerId |   Survived |   Pclass | Name                                                | Sex    |   Age |   SibSp |   Parch | Ticket           |    Fare | Cabin   | Embarked   |
|---------------+------------+----------+-----------------------------------------------------+--------+-------+---------+---------+------------------+---------+---------+------------|
|             1 |          0 |        3 | Braund, Mr. Owen Harris                             | male   |    22 |       1 |       0 | A/5 21171        |  7.25   | nan     | S          |
|             2 |          1 |        1 | Cumings, Mrs. John Bradley (Florence Briggs Thayer) | female |    38 |       1 |       0 | PC 17599         | 71.28

In [14]:
titanic_pq

DataFrame[PassengerId: bigint, Survived: bigint, Pclass: bigint, Name: string, Sex: string, Age: double, SibSp: bigint, Parch: bigint, Ticket: string, Fare: double, Cabin: string, Embarked: string]

In [15]:
vd.get_pandas_schemaStr(titanic_df)

'DataFrame[PassengerId: int64, Survived: int64, Pclass: int64, Name: string, Sex: string, Age: float64, SibSp: int64, Parch: int64, Ticket: string, Fare: float64, Cabin: string, Embarked: string]'

In [16]:
titanic_pq.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25|  nan|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925|  nan|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05|  nan|       S|
|          6|       0|     3|    Moran, Mr. James|  male| NaN|    0|    0|      

In [17]:
vd.show_pandas_as_table(titanic_df, num_rows=20)

+---------------+------------+----------+---------------------------------------------------------+--------+-------+---------+---------+------------------+---------+---------+------------+
|   PassengerId |   Survived |   Pclass | Name                                                    | Sex    |   Age |   SibSp |   Parch | Ticket           |    Fare | Cabin   | Embarked   |
|---------------+------------+----------+---------------------------------------------------------+--------+-------+---------+---------+------------------+---------+---------+------------|
|             1 |          0 |        3 | Braund, Mr. Owen Harris                                 | male   |    22 |       1 |       0 | A/5 21171        |  7.25   | nan     | S          |
|             2 |          1 |        1 | Cumings, Mrs. John Bradley (Florence Briggs Thayer)     | female |    38 |       1 |       0 | PC 17599         | 71.2833 | C85     | C          |
|             3 |          1 |        3 | Heikkinen, Mi

In [18]:
titanic_df.head(20)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,nan,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,nan,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,nan,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,nan,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,nan,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,nan,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,nan,C


In [19]:
vd.tblt_ocurrences([citizens_df.City])
print('\n')
vd.tblt_ocurrences([citizens_df.City, citizens_df.Age], conditionated=True)

Value          Count
-----------  -------
New York           2
Los Angeles        1
Chicago            1
Houston            1


Value        Condition      Count
-----------  -----------  -------
Chicago      Age=32             1
Houston      Age=44             1
Los Angeles  Age=22             1
New York     Age=24             1
New York     Age=27             1


In [20]:
vd.tblt_concentrations(citizens_df, 'Age', [0,20,40,60])

Concentration by threshold:
  Threshold for Age    Concentration
-------------------  ---------------
                  0                5
                 20                5
                 40                1
                 60                0


In [21]:
vd.unique_values(citizens_df)

Unique values per variable:

Name: ['Alice', 'Bob', 'Charlie', 'David']
----------
Age: [24, 27, 22, 32, 44]
----------
City: ['New York', 'Los Angeles', 'Chicago', 'Houston']
----------


In [22]:
vd.equal_df(citizens_df, citizens2_df, aggregated=False)
print('\n')
vd.equal_df(citizens_df, citizens2_df, aggregated=True)

False


    Name    Age   City
0  False  False  False
1  False  False  False
2  False  False  False
3  False   True  False
4   True  False  False


In [23]:
vd.equal_df_mult({'titanic': titanic_pq, 'titanic_slice1': titanic_pq_slice1, 'titanic_copy': titanic_pq_copy}, df_comparison = 'titanic', row_count = False)

--- Comparing [titanic] and titanic
— Skipping self reference

--- Comparing [titanic] and titanic_slice1
E The DataFrames have a different number of columns, operation cannot be performed

--- Comparing [titanic] and titanic_copy
X titanic and titanic_copy have a different shape:
titanic                  -> Columns: 12 x Rows: Omitted | Different Rows Records: 1
titanic_copy             -> Columns: 12 x Rows: Omitted | Different Rows Records: 1



In [24]:
print(vd.b('pd case:\n'))
vd.check_variables({'titanic': titanic_df, 'titanic_slice1': titanic_df_slice1, 'titanic_slice2':titanic_df_slice2}, df_comparison=None)
print('\n')
vd.check_variables({'titanic': titanic_df, 'titanic_slice1': titanic_df_slice1, 'titanic_slice2':titanic_df_slice2}, df_comparison='titanic')

pd case:

Mode: Comparing all variables
+-----------------------------------+-----------------------------------+-----------------------------------+
|titanic (x=12)                     |titanic_slice1 (x=8)               |titanic_slice2 (x=4)               |
+-----------------------------------+-----------------------------------+-----------------------------------+
|Age                                |Age                                |Age                                |
|Cabin                              |...                                |...                                |
|Embarked                           |...                                |...                                |
|Fare                               |...                                |...                                |
|Name                               |Name                               |...                                |
|Parch                              |Parch                              |Parch  

In [25]:
print(vd.b('pq case:\n'))
vd.check_variables({'citizensN': citizensN_pq, 'citizensNN': citizensNN_pq}, df_comparison=None)
print('\n')
vd.check_variables({'citizensN': citizensN_pq, 'citizensNN': citizensNN_pq}, df_comparison='citizensN')

pq case:

Mode: Comparing all variables
+-----------------------------------+-----------------------------------+
|citizensN (x=4)                    |citizensNN (x=5)                   |
+-----------------------------------+-----------------------------------+
|age                                |age                                |
|city                               |city                               |
|name                               |name                               |
|number                             |number                             |
|...                                |number2                            |
+-----------------------------------+-----------------------------------+


Mode: Comparing to "citizensN" variables
+-----------------------------------+-----------------------------------+
|citizensN (x=4)                    |citizensNN (x=5)                   |
+-----------------------------------+-----------------------------------+
|age                         

### - auxiliary

In [26]:
print(vd.check_type(pd_series))
print(vd.check_type(np_array))
print(vd.check_type(titanic_df))
print(vd.check_type(titanic_pq))
print(vd.check_type(py_list))
print(vd.check_type(citizens_dict))

Pandas Series
NumPy Array
Pandas DataFrame
Spark DataFrame
List
Dictionary


In [27]:
vd.path_correction('.\\.\\.\\')

'./././'

In [28]:
vd.now_timedelta()

datetime.datetime(2024, 11, 10, 18, 44, 19, 283905, tzinfo=<DstTzInfo 'America/Bogota' -05-1 day, 19:00:00 STD>)

### - utils

In [29]:
#vd.save_file() !TO DO

In [30]:
# def read_file() !TO DO

In [31]:
vd.check_directory('./build')
vd.check_directory('./build_')

✓ The directory './build' exists
X The directory './build_' does not exist


In [32]:
vd.get_directories('./')

'./'
Contents of the current directory:
build
jupyter_testing.ipynb
make.bat
Makefile
open_vd.bat
setup.py
source
valdata
valdata.code-workspace
valdata.log
venv
__pycache__


In [33]:
vd.get_modules()

Loaded modules:

IPython
IPython.core
IPython.core.alias
IPython.core.application
IPython.core.async_helpers
IPython.core.autocall
IPython.core.builtin_trap
IPython.core.compilerop
IPython.core.completer
IPython.core.completerlib
IPython.core.crashhandler
IPython.core.debugger
IPython.core.display
IPython.core.display_functions
IPython.core.display_trap
IPython.core.displayhook
IPython.core.displaypub
IPython.core.error
IPython.core.events
IPython.core.excolors
IPython.core.extensions
IPython.core.formatters
IPython.core.getipython
IPython.core.guarded_eval
IPython.core.history
IPython.core.hooks
IPython.core.inputtransformer2
IPython.core.interactiveshell
IPython.core.latex_symbols
IPython.core.logger
IPython.core.macro
IPython.core.magic
IPython.core.magic_arguments
IPython.core.magics
IPython.core.magics.ast_mod
IPython.core.magics.auto
IPython.core.magics.basic
IPython.core.magics.code
IPython.core.magics.config
IPython.core.magics.display
IPython.core.magics.execution
IPython.core

In [34]:
vd.add_module_path('.\directory_to_other_library')

In [35]:
vd.get_hadoop_version()

Python Version: 3.9.7 (tags/v3.9.7:1016ef3, Aug 30 2021, 20:19:38) [MSC v.1929 64 bit (AMD64)]
Hadoop Version: Hadoop might not be installed or properly loaded.
Spark Version: 3.1.1
PySpark Version: 3.1.1


### - visuals

In [36]:
print('-' * vd.padding10)
print('-' * vd.padding25)
print('-' * vd.padding35)

----------
-------------------------
-----------------------------------


In [37]:
print('Standard Text')
print(vd.b('Bold text'))
print(vd.b_bl('Bold blue text'))
print(vd.b_re('Bold red text'))
print(vd.b_gr('Bold green text'))

Standard Text
Bold text
Bold blue text
Bold red text
Bold green text


In [38]:
vd.set_start('Set start')
# !styling ONLY applies for jupyter environments

Set start
Nov-10-2024 06:44:20 PM
Python: 3.9.7 (tags/v3.9.7:1016ef3, Aug 30 2021, 20:19:38) [MSC v.1929 64 bit (AMD64)]
